<a href="https://colab.research.google.com/github/ileocho/deepfake_s2d/blob/main/Music_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/gdrive/')
import sys #global location
location="/content/gdrive/My Drive/TP5/" #
sys.path.append(location) # The location of the .ipynb file.  

Mounted at /content/gdrive/


# <font color='black'>Deep Neural Networks</font>

---
---
<figure>
<center>
<img src='https://drive.google.com/uc?id=1u-iZKwzco1L8a3gFFLMXbSOU_DsdNZqo' width="300" align="center" />
</center>
</figure>

> Year: **2022**

> Version: **2.1**

- You need to solve these exercises in groups of two or three people --> A group of one person will be penalized -3 points so please work in a group.  
- You are required to upload one solution per group on Moodle before the deadline. 

Please format the name of the file as TP5_NAME1_NAME2.ipynb or TP5_NAME1_NAME2_NAME3.ipynb

Good luck!
# Use a LSTM Network to generate music

In this exercsie you will practise the sequence models seen in class. Notably, you will explore the use of a LSTM network for music generation.

In [6]:
%cd /content/gdrive/My Drive/TP5/

/content/gdrive/My Drive/TP5


In [8]:
import IPython
import sys
from music21 import *
import numpy as np
from grammar import *
from qa import *
from preprocess import *
from music_utils import *
from data_utils import *
from keras.models import load_model, Model
from keras.layers import Dense, Activation, Dropout, Input, LSTM, Reshape, Lambda, RepeatVector
from keras.initializers import glorot_uniform
from keras.utils import to_categorical
from keras.optimizers import Adam
from keras import backend as K

In the provided dataset each value corresponds to a note.
The music generation system uses 78 notes. Start by running the following cell that loads the music data and converts it into values.

In [9]:
X, Y, n_values, indices_values = load_music_utils()

- `X`: This is an (m, $T_x$, 78) dimensional array. The input is one of 78 different possible notes, represented as a one-hot vector. Thus for example, X[i,t,:] is a one-hot vector representating the value of the i-th example at time t. The 

- `Y`: is an (m, $T_x$, 78) dimensional array.

- `n_values`: This is 78. 

- `indices_values`: python dictionary from 0-77 notes.

You will now build and train a model that will learn the desired musical patterns from the data. In order to do it, you will need to define a model that takes in the matrices X and Y of shape. We will use an LSTM with 64 dimensional hidden states.


In [10]:
n_a = 64 


Now you are going to create a Keras model with multiple inputs and outputs. Remark that at test time we don't know all the values of $x^{\langle t\rangle}$ in advance, we only generate one at a time using $x^{\langle t\rangle} = y^{\langle t-1 \rangle}$. You'll need to implement a for-loop to iterate over the different time steps. 

The function `djmodel()` will call the LSTM layer $T_x$ times using a for-loop. All $T_x$ blocks have the same, i.e. $T_x$ steps share the same weights. 

Therefore you have to
1. Define the layers as global variables.
2. Call these layers when propagating the input.

The layers are defined as global variables in the following cell.

In [73]:
n_values=78
reshapor = Reshape((1, n_values))                        # Used in Step 2.B of djmodel(), below
LSTM_cell = LSTM(n_a, return_state = True)         # Used in Step 2.C
densor = Dense(n_values, activation='softmax')     # Used in Step 2.D

You have defined `reshapor`, `LSTM_cell` and `densor` as layer objects. And they can be used in `djmodel()`. 

In order to propagate a Keras tensor X through one layer, use `layer_object(X)` (or `layer_object([X,Y])` if it requires multiple inputs.). For example, `reshapor(X)`.

Implement `djmodel()`. You will need to carry out 2 steps:

1. Create an empty list "outputs" to save the outputs of the LSTM Cell at every time step.
2. Loop for `t in range(T_x):`

    A. Select the "t"th time-step vector from X with:
```    
           x = Lambda(lambda x: x[:,t,:])(X)
``` 

    B. Reshape x to (1,78) using `reshapor()`.

    C. Now you can run x through one step of LSTM_cell. The previous step's hidden state $a$ and cell state $c$ are used for initialization according to:
```python
a, _, c = LSTM_cell(input_x, initial_state=[previous hidden state, previous cell state])
```

    D. Propagate the LSTM's output activation value through a dense+softmax layer using `densor`. 
    
    E. Append the predicted value to the list of "outputs"

In [74]:
def djmodel(Tx, n_a, n_values):
    """
    Implement the model
    Arguments:
    Tx -- length of the sequence in a corpus
    n_a -- the number of activations used in our model
    n_values -- number of unique values in the music data
    Returns:
    model -- a keras model. Here an LSTM model. 
    """

    # Define the input of your model with a shape
    X = Input(shape=(Tx, n_values))

    # Define s0, initial hidden state for the decoder LSTM
    a0 = Input(shape=(n_a,), name='a0')
    c0 = Input(shape=(n_a,), name='c0')
    a = a0
    c = c0

    ### START CODE HERE ###
    # Step 1: Create empty list to append the outputs while you iterate (≈1 line)
    outputs = []

    # Step 2: Loop
    for t in range(Tx):

        # Step 2.A: select the "t"th time step vector from X.  (≈1 line)
        ts_vector = Lambda(lambda x: x[:, t, :])(X)

        # Step 2.B: Use reshapor to reshape x to be (1, n_values) (≈1 line)
        ts_vector = reshapor(ts_vector)

        # Step 2.C: Perform one step of the LSTM_cell (≈1 line)
        a, _, c = LSTM_cell(ts_vector, initial_state=[a, c])

        # Step 2.D: Apply densor to the hidden state output of LSTM_Cell (≈1 line)
        out = densor(a)

        # Step 2.E: add the output to "outputs" (≈1 line)
        outputs.append(out)

    # Step 3: Create model instance (≈1 line)
    model = Model(inputs=[X, a0, c0], outputs=outputs)

    ### END CODE HERE ###

    return model

Define and compile the model in Keras. 

In [75]:
model = djmodel(Tx = 30 , n_a = 64, n_values = 78)

In [77]:
opt = Adam(learning_rate = 0.01, beta_1 = 0.9, beta_2 = 0.999, decay = 0.01) #Set the hyperparameters related to Adam optimizer.
model.compile(optimizer='adam', loss="categorical_crossentropy",metrics=['accuracy'])

Initialize `a0` and `c0` for the LSTM's initial state to be zero. 

In [78]:
m = 60
a0 = np.zeros((m, n_a))
c0 = np.zeros((m, n_a))

Next cell will fit the model. `Y` has been converted to a list with 30 items. The model is trained for 100 epochs.

In [79]:
model.fit([X, a0, c0], list(Y), epochs=100)

Epoch 1/100
2/2 [==============================] - 45s 62ms/step - loss: 126.2985 - dense_1_loss: 4.3598 - dense_1_1_loss: 4.3544 - dense_1_2_loss: 4.3643 - dense_1_3_loss: 4.3552 - dense_1_4_loss: 4.3457 - dense_1_5_loss: 4.3662 - dense_1_6_loss: 4.3742 - dense_1_7_loss: 4.3392 - dense_1_8_loss: 4.3451 - dense_1_9_loss: 4.3668 - dense_1_10_loss: 4.3560 - dense_1_11_loss: 4.3491 - dense_1_12_loss: 4.3557 - dense_1_13_loss: 4.3533 - dense_1_14_loss: 4.3429 - dense_1_15_loss: 4.3766 - dense_1_16_loss: 4.3557 - dense_1_17_loss: 4.3423 - dense_1_18_loss: 4.3745 - dense_1_19_loss: 4.3595 - dense_1_20_loss: 4.3509 - dense_1_21_loss: 4.3466 - dense_1_22_loss: 4.3392 - dense_1_23_loss: 4.3753 - dense_1_24_loss: 4.3421 - dense_1_25_loss: 4.3422 - dense_1_26_loss: 4.3444 - dense_1_27_loss: 4.3487 - dense_1_28_loss: 4.3727 - dense_1_29_loss: 0.0000e+00 - dense_1_accuracy: 0.0167 - dense_1_1_accuracy: 0.0000e+00 - dense_1_2_accuracy: 0.0000e+00 - dense_1_3_accuracy: 0.0000e+00 - dense_1_4_accuracy

## Music generation

You will now implement a function to sample a sequence of musical values. 

Step 2.A: Use `LSTM_Cell`, which inputs the previous step's `c` and `a` to generate the current step's `c` and `a`. 

Step 2.B: Use `densor` to compute a softmax on `a` to get the output for the current step `y`. 

Step 2.C: Save the output you have just generated by appending it to `outputs`.

Step 2.D: Sample x to the be "out"'s one-hot version to pass it to the next LSTM's step with: 

```python
    x = Lambda(one_hot)(out) 
```

In [84]:
def music_inference_model(LSTM_cell, densor, n_values=78, n_a=64, Ty=100):
    """
    Uses the trained "LSTM_cell" and "densor" from model() to generate a sequence of values.
    Arguments:
    LSTM_cell -- the trained "LSTM_cell" from model(), Keras layer object
    densor -- the trained "densor" from model(), Keras layer object
    n_values -- integer, umber of unique values
    n_a -- number of units in the LSTM_cell
    Ty -- integer, number of time steps to generate
    Returns:
    inference_model -- Keras model instance
    """

    # Define the input of your model with a shape
    x0 = Input(shape=(1, n_values))

    # Define s0, initial hidden state for the decoder LSTM
    a0 = Input(shape=(n_a,), name='a0')
    c0 = Input(shape=(n_a,), name='c0')
    a = a0
    c = c0
    x = x0

    ### START CODE HERE ###
    # Step 1: Create an empty list of "outputs" to later store your predicted values (≈1 line)
    outputs = []

    # Step 2: Loop over Ty and generate a value at every time step
    for t in range(Ty):
        # Step 2.A: Perform one step of LSTM_cell (≈1 line)
        a, _, c = LSTM_cell(x, initial_state=[a, c])

        # Step 2.B: Apply Dense layer to the hidden state output of the LSTM_cell (≈1 line)
        out = densor(a)

        # Step 2.C: Append the prediction "out" to "outputs". out.shape = (None, 78) (≈1 line)
        outputs.append(out)

        # Step 2.D: Select the next value according to "out", and set "x" to be the one-hot representation of the
        #           selected value, which will be passed as the input to LSTM_cell on the next step. We have provided
        #           the line of code you need to do this.  (≈1 line)
        x = Lambda(one_hot)(out) 

    # Step 3: Create model instance with the correct "inputs" and "outputs" (≈1 line)
    inference_model = Model(inputs=[x0, a0, c0], outputs=outputs)
    ### END CODE HERE ###

    return inference_model

In [85]:
inference_model = music_inference_model(LSTM_cell, densor, n_values = 78, n_a = 64, Ty = 50)

x_initializer = np.zeros((1, 1, 78))
a_initializer = np.zeros((1, n_a))
c_initializer = np.zeros((1, n_a))

Implement the function `predict_and_sample()`.The arguments of this function are [x_initializer, a_initializer, c_initializer]. Afterwards, in order to predict the corresponding output to this input, you should:
1. Use the inference model to predict an output for a given your set of inputs. The output `pred` is a list of length $T_y$ where each element is a numpy-array of shape (1, n_values).
2. Convert `pred` into a numpy array of $T_y$ indices. To compute each index use the `argmax` of an element of the `pred` list.
3. The indices have to be converted into their one-hot vector representations.

In [112]:
def predict_and_sample(inference_model, x_initializer = x_initializer, a_initializer = a_initializer, 
                       c_initializer = c_initializer):
    """
    Predicts the next value of values using the inference model.
    
    Arguments:
    inference_model -- Keras model instance for inference time
    x_initializer -- numpy array of shape (1, 1, 78), one-hot vector initializing the values generation
    a_initializer -- numpy array of shape (1, n_a), initializing the hidden state of the LSTM_cell
    c_initializer -- numpy array of shape (1, n_a), initializing the cell state of the LSTM_cel
    
    Returns:
    results -- numpy-array of shape (Ty, 78), matrix of one-hot vectors representing the values generated
    indices -- numpy-array of shape (Ty, 1), matrix of indices representing the values generated
    """
    
    ### START CODE HERE ### 
    # Step 1: Use your inference model to predict (inference_model.predict)
    # an output sequence given x_initializer, a_initializer and c_initializer.  (≈1 line)
    pred = inference_model.predict([x_initializer, a_initializer, c_initializer])

    # Step 2: Convert "pred" into an np.array() of indices with the maximum probabilities (≈1 line) 
    # or use np.argmax (≈1 line)
    indices = np.argmax(pred, axis=-1)
    

    # Step 3: Use to_categorical from keras.utils (already imported)
    # to convert indices to one-hot vectors, the shape of the results should be (1, )
    results = to_categorical(indices, num_classes=78)
    
    ### END CODE HERE ###
    
    return results, indices

In [114]:
results, indices = predict_and_sample(inference_model, x_initializer, a_initializer, c_initializer)
print("np.argmax(results[12]) =", np.argmax(results[12]))
print("np.argmax(results[17]) =", np.argmax(results[17]))
print("list(indices[12:18]) =", list(indices[12:18]))

1/1 [==============================] - 0s 45ms/step
np.argmax(results[12]) = 0
np.argmax(results[17]) = 77
list(indices[12:18]) = [array([0]), array([0]), array([0]), array([0]), array([0]), array([77])]


Run the following cell to generate music and record it into  `out_stream`

In [115]:
out_stream = generate_music(inference_model)

Predicting new values for different set of chords.
1/1 [==============================] - 0s 45ms/step
Generated 51 sounds using the predicted values for the set of chords ("1") and after pruning
1/1 [==============================] - 0s 43ms/step
Generated 49 sounds using the predicted values for the set of chords ("2") and after pruning
1/1 [==============================] - 0s 54ms/step
Generated 51 sounds using the predicted values for the set of chords ("3") and after pruning
1/1 [==============================] - 0s 46ms/step
Generated 50 sounds using the predicted values for the set of chords ("4") and after pruning
1/1 [==============================] - 0s 44ms/step
Generated 51 sounds using the predicted values for the set of chords ("5") and after pruning
Your generated music is saved in output/my_music.midi


To listen to your music, click File->Open... Then go to "output/" and download "my_music.midi". You can play it on your computer.

# A) Retrain your model with a learning rate larger than 0.01.


In [116]:
model = djmodel(Tx = 30 , n_a = 64, n_values = 78)
opt = Adam(learning_rate = 0.1, beta_1 = 0.9, beta_2 = 0.999, decay = 0.01) #Set the hyperparameters related to Adam optimizer.
model.compile(optimizer='adam', loss="categorical_crossentropy",metrics=['accuracy'])

m = 60
a0 = np.zeros((m, n_a))
c0 = np.zeros((m, n_a))

model.fit([X, a0, c0], list(Y), epochs=100)

Epoch 1/100
2/2 [==============================] - 42s 67ms/step - loss: 93.6988 - dense_1_loss: 4.1679 - dense_1_1_loss: 4.0316 - dense_1_2_loss: 3.7480 - dense_1_3_loss: 3.6538 - dense_1_4_loss: 3.2915 - dense_1_5_loss: 3.4087 - dense_1_6_loss: 3.3396 - dense_1_7_loss: 3.1101 - dense_1_8_loss: 3.1507 - dense_1_9_loss: 3.0076 - dense_1_10_loss: 3.0644 - dense_1_11_loss: 3.1616 - dense_1_12_loss: 2.9611 - dense_1_13_loss: 3.0268 - dense_1_14_loss: 3.0951 - dense_1_15_loss: 3.2457 - dense_1_16_loss: 3.1473 - dense_1_17_loss: 3.0899 - dense_1_18_loss: 2.9635 - dense_1_19_loss: 3.0406 - dense_1_20_loss: 3.1482 - dense_1_21_loss: 2.9769 - dense_1_22_loss: 3.2216 - dense_1_23_loss: 3.2204 - dense_1_24_loss: 3.3244 - dense_1_25_loss: 2.9964 - dense_1_26_loss: 3.0109 - dense_1_27_loss: 3.1361 - dense_1_28_loss: 2.9582 - dense_1_29_loss: 0.0000e+00 - dense_1_accuracy: 0.1167 - dense_1_1_accuracy: 0.0500 - dense_1_2_accuracy: 0.1667 - dense_1_3_accuracy: 0.2167 - dense_1_4_accuracy: 0.2500 - de

In [117]:
inference_model = music_inference_model(LSTM_cell, densor, n_values = 78, n_a = 64, Ty = 50)

x_initializer = np.zeros((1, 1, 78))
a_initializer = np.zeros((1, n_a))
c_initializer = np.zeros((1, n_a))

out_stream = generate_music(inference_model)

Predicting new values for different set of chords.
1/1 [==============================] - 15s 15s/step
Generated 51 sounds using the predicted values for the set of chords ("1") and after pruning
1/1 [==============================] - 0s 39ms/step
Generated 46 sounds using the predicted values for the set of chords ("2") and after pruning
1/1 [==============================] - 0s 41ms/step
Generated 50 sounds using the predicted values for the set of chords ("3") and after pruning
1/1 [==============================] - 0s 47ms/step
Generated 49 sounds using the predicted values for the set of chords ("4") and after pruning
1/1 [==============================] - 0s 45ms/step
Generated 49 sounds using the predicted values for the set of chords ("5") and after pruning
Your generated music is saved in output/my_music.midi


# B) Retrain your model with a learning rate smaller than 0.01.


In [118]:
model = djmodel(Tx = 30 , n_a = 64, n_values = 78)
opt = Adam(learning_rate = 0.001, beta_1 = 0.9, beta_2 = 0.999, decay = 0.01) #Set the hyperparameters related to Adam optimizer.
model.compile(optimizer='adam', loss="categorical_crossentropy",metrics=['accuracy'])

m = 60
a0 = np.zeros((m, n_a))
c0 = np.zeros((m, n_a))

model.fit([X, a0, c0], list(Y), epochs=100)

Epoch 1/100
2/2 [==============================] - 41s 64ms/step - loss: 82.5634 - dense_1_loss: 3.9475 - dense_1_1_loss: 3.7180 - dense_1_2_loss: 3.2773 - dense_1_3_loss: 3.0274 - dense_1_4_loss: 2.7015 - dense_1_5_loss: 2.7917 - dense_1_6_loss: 2.8502 - dense_1_7_loss: 2.7121 - dense_1_8_loss: 2.6554 - dense_1_9_loss: 2.6756 - dense_1_10_loss: 2.6865 - dense_1_11_loss: 2.7431 - dense_1_12_loss: 2.5673 - dense_1_13_loss: 2.6270 - dense_1_14_loss: 2.7864 - dense_1_15_loss: 2.8886 - dense_1_16_loss: 2.8296 - dense_1_17_loss: 2.8929 - dense_1_18_loss: 2.6522 - dense_1_19_loss: 2.7146 - dense_1_20_loss: 2.8127 - dense_1_21_loss: 2.6466 - dense_1_22_loss: 2.9046 - dense_1_23_loss: 2.8749 - dense_1_24_loss: 2.9855 - dense_1_25_loss: 2.6783 - dense_1_26_loss: 2.6778 - dense_1_27_loss: 2.7593 - dense_1_28_loss: 2.4788 - dense_1_29_loss: 0.0000e+00 - dense_1_accuracy: 0.0833 - dense_1_1_accuracy: 0.0500 - dense_1_2_accuracy: 0.2167 - dense_1_3_accuracy: 0.2167 - dense_1_4_accuracy: 0.2833 - de

In [119]:
inference_model = music_inference_model(LSTM_cell, densor, n_values = 78, n_a = 64, Ty = 50)

x_initializer = np.zeros((1, 1, 78))
a_initializer = np.zeros((1, n_a))
c_initializer = np.zeros((1, n_a))

out_stream = generate_music(inference_model)

Predicting new values for different set of chords.
1/1 [==============================] - 15s 15s/step
Generated 50 sounds using the predicted values for the set of chords ("1") and after pruning
1/1 [==============================] - 0s 42ms/step
Generated 48 sounds using the predicted values for the set of chords ("2") and after pruning
1/1 [==============================] - 0s 41ms/step
Generated 49 sounds using the predicted values for the set of chords ("3") and after pruning
1/1 [==============================] - 0s 42ms/step
Generated 47 sounds using the predicted values for the set of chords ("4") and after pruning
1/1 [==============================] - 0s 45ms/step
Generated 50 sounds using the predicted values for the set of chords ("5") and after pruning
Your generated music is saved in output/my_music.midi
